In [1]:
import logging

import numpy as np
import optimistix as optx
from atmodeller import (
    EquilibriumModel,
    Planet,
    ChemicalSpecies,
    SpeciesNetwork,
    debug_logger,
    earth_oceans_to_hydrogen_mass,
    SolverParameters,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm

from scipy.interpolate import interp1d
import os

from matplotlib.ticker import ScalarFormatter

formatter = ScalarFormatter(useMathText=False)
formatter.set_scientific(False)
formatter.set_useOffset(False)
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

logger = debug_logger() #you can dump it in file 
#logger.setLevel(logging.INFO) #you could set it to error

logger.setLevel(logging.DEBUG)  
##rebase for atmodeller git pull, git re base (you might also have to git pull when paralell)

Atmodeller initialized with double precision (float64)


# Set Elemental Abundances

In [2]:
# Palme and O'Neill (2014) Treatise on Geochemistry - Table 3

SiO2_mantlemasspercent_palme14: float = 45.4
total_mantlemasspercent_palme14: float = 98.41
core_mass_fraction: float = 0.327  # typical values used are between 0.325 - 0.33
Si_massfraction_palme14: float = round(
    SiO2_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction)
    / (28.0855 + 2 * 15.999)
    * 28.0855,
    3,
)

O_massfraction_palme14: float = Si_massfraction_palme14 / 28.0855 * 2 * 15.999


# Lodders et al. (2009) Springer Book Chapter - Table 8 (in wt%)

H_masspercent_lodders09: float = 73.9
He_masspercent_lodders09: float = 24.69
C_masspercent_lodders09: float = 0.22
N_masspercent_lodders09: float = 0.07
O_masspercent_lodders09: float = 0.63
Si_masspercent_lodders09: float = 0.07


# Lodders et al. (2009) Springer Book Chapter - Table 6 (log-normalalized abundances relative to H)

H_logN: float = 12
He_logN: float = 10.93
C_logN: float = 8.39
N_logN: float = 7.86
O_logN: float = 8.73
Si_logN: float = 7.53

Primitive composition of the Earth's mantle \
Table 3 - Palme and O'Neill (2014) Treatise on Geochemistry

| Component  | Mass % |
|------------|--------|
| MgO        | 36.77  |
| Al2O3      | 4.49   |
| SiO2       | 45.4   |
| CaO        | 3.65   |
| FeO(t)     | 8.1    |
| Total      | 98.41  |
| Mg#        | 0.890  |


Present-day solar composition \
Table 8 - Lodders et al. (2009) Springer book chapter 

| Element    | Mass % |
|------------|--------|
| H (=X)     | 73.9   |
| He (=Y)    | 24.69  |
| O          | 0.63   |
| C          | 0.22   |
| Ne         | 0.17   |
| Fe         | 0.12   |
| N          | 0.07   |
| Si         | 0.07   |
| Mg         | 0.06   |
| S          | 0.03   |
| others     | 0.04   |
| total (=Z) | 1.41   |

Table 6 - Lodders et al. (2009) Springer book chapter 

| Element | A (log N(H) = 12) |
|---------|-------------------|
| H       | 12                |
| He      | 10.93             |
| C       | 8.39              |
| N       | 7.86              |
| O       | 8.73              |
| Ne      | 8.05              |
| Na      | 6.29              |
| Mg      | 7.54              |
| Al      | 6.46              |
| Si      | 7.53              |
| P       | 5.45              |
| S       | 7.16              |
| Cl      | 5.25              |
| Ar      | 6.5               |
| K       | 5.11              |
| Ca      | 6.31              |
| Ti      | 4.93              |
| V       | 3.99              |
| Fe      | 7.46              |


# Case 0

### Species

In [3]:
eos_models = get_eos_models()
sol_models = get_solubility_models()

H2O_g = ChemicalSpecies.create_gas("H2O")
H2O_gs = ChemicalSpecies.create_gas("H2O", solubility=sol_models["H2O_peridotite_sossi23"])
H2O_rgs = ChemicalSpecies.create_gas(
    "H2O",
    activity=eos_models["H2O_cork_holland98"],
    solubility=sol_models["H2O_peridotite_sossi23"],
)

H2_g = ChemicalSpecies.create_gas("H2")
H2_gs = ChemicalSpecies.create_gas("H2", solubility=sol_models["H2_basalt_hirschmann12"])
H2_rgs = ChemicalSpecies.create_gas(
    "H2", activity=eos_models["H2_chabrier21"], solubility=sol_models["H2_basalt_hirschmann12"]
)

O2_g = ChemicalSpecies.create_gas("O2")
O2_rg = ChemicalSpecies.create_gas("O2", activity=eos_models["O2_cs_shi92"])

OSi_g = ChemicalSpecies.create_gas("OSi")
OSi_rg = ChemicalSpecies.create_gas("OSi", activity=eos_models["OSi_rk49_connolly16"])

H4Si_g = ChemicalSpecies.create_gas("H4Si")
H4Si_rg = ChemicalSpecies.create_gas("H4Si", activity=eos_models["H4Si_wang18"])

O2Si_l = ChemicalSpecies.create_condensed("O2Si", state="l")
O2Si_bqz = ChemicalSpecies.create_condensed("O2Si", state="bqz")
CO2_g = ChemicalSpecies.create_gas("CO2")
CO2_gs = ChemicalSpecies.create_gas("CO2", solubility=sol_models["CO2_basalt_dixon95"])
CO2_rgs = ChemicalSpecies.create_gas(
    "CO2", solubility=sol_models["CO2_basalt_dixon95"], activity=eos_models["CO2_cs_shi92"]
)
C_cr = ChemicalSpecies.create_condensed("C", state="cr")

#
CO_g = ChemicalSpecies.create_gas("CO")
CO_gs = ChemicalSpecies.create_gas("CO", solubility=sol_models["CO_basalt_yoshioka19"])
CO_rgs = ChemicalSpecies.create_gas(
    "CO", solubility=sol_models["CO_basalt_yoshioka19"], activity=eos_models["CO_cs_shi92"]
)
#
CH4_g = ChemicalSpecies.create_gas("CH4")
CH4_gs = ChemicalSpecies.create_gas("CH4", solubility=sol_models["CH4_basalt_ardia13"])
CH4_rgs = ChemicalSpecies.create_gas(
    "CH4", solubility=sol_models["CH4_basalt_ardia13"], activity=eos_models["CH4_cs_shi92"]
)
#
N2_g = ChemicalSpecies.create_gas("N2")
N2_gs = ChemicalSpecies.create_gas("N2", solubility=sol_models["N2_basalt_libourel03"])
N2_rgs = ChemicalSpecies.create_gas(
    "N2",solubility=sol_models["N2_basalt_libourel03"],activity=eos_models["N2_cs_saxena87"]
)

NH3_g = ChemicalSpecies.create_gas("NH3")
NH3_rg = ChemicalSpecies.create_gas("H3N", activity=eos_models["H3N_rk49_reid87"])

He_g = ChemicalSpecies.create_gas("He")
He_gs = ChemicalSpecies.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
He_rgs = ChemicalSpecies.create_gas(
    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
)

species_HHeCNOSi_magma_nosol_ideal = SpeciesNetwork(
    (
        H2O_g, 
        H2_g, 
        O2_g, 
        OSi_g, 
        H4Si_g, 
        O2Si_l, 
#        C_cr,
        N2_g, 
        NH3_g, 
        CH4_g, 
        CO_g, 
        CO2_g, 
        He_g
        )
)
species_HHeCNOSi_magma_sol_ideal = SpeciesNetwork(
    (
        H2O_gs, 
        H2_gs, 
        O2_g, 
        OSi_g, 
        H4Si_g, 
        O2Si_l,
#        C_cr,
        N2_gs, 
        NH3_g, 
        CH4_gs, 
        CO_gs, 
        CO2_gs, 
        He_gs
        )
)
species_HHeCNOSi_magma_sol_real = SpeciesNetwork(
    (
        H2O_rgs,
        H2_rgs,
        O2_rg,
        OSi_rg,
        H4Si_rg,
        O2Si_l,
#        C_cr,
        N2_rgs,
        NH3_rg,
        CH4_rgs, 
        CO_rgs, 
        CO2_rgs, 
        He_rgs
    )
)

### Solve system with mass constraints

In [14]:
# Overall structure 
R_earth = 6.371e6
R_uranus = 3.939 * R_earth
M_earth = 5.972e24 #kg
M_uranus = 14.536 * M_earth

# Bulk Envelope
M_envelope = np.logspace(-3, -2, 10) * M_uranus

# Rocky Core 
M_core = M_uranus - M_envelope
core_mantle_melt_fraction = 1.0
R_core = 1.02 * (M_core / M_earth)**0.252 * R_earth # M-R relation from Hakim et al. (2018)
core_temperature = 4400


# Try two endmember metallicity cases
init_metallicitys = [1, 100]  # metallicity in x solar units

# Initialize EquilibriumModel objects outside of the loop 
model_magma_nosol_ideal = EquilibriumModel(species_HHeCNOSi_magma_nosol_ideal)
model_magma_sol_ideal = EquilibriumModel(species_HHeCNOSi_magma_sol_ideal)
model_magma_sol_real = EquilibriumModel(species_HHeCNOSi_magma_sol_real)

for init_metallicity in init_metallicitys:

    planet = Planet(
        temperature=core_temperature,
        planet_mass=M_core,
        mantle_melt_fraction=1,
        surface_radius=R_core,
    )

    # Lodders et al. (2009) Springer book chapter - Table 8
    scale_percent_metallicity = H_masspercent_lodders09 + He_masspercent_lodders09 + init_metallicity * (
        C_masspercent_lodders09 + N_masspercent_lodders09 + 
        Si_masspercent_lodders09 + 
        O_masspercent_lodders09)

    h_kg = M_envelope * H_masspercent_lodders09 / scale_percent_metallicity
    he_kg = M_envelope * He_masspercent_lodders09 / scale_percent_metallicity
    c_kg = M_envelope * init_metallicity * C_masspercent_lodders09 / scale_percent_metallicity
    n_kg = M_envelope * init_metallicity * N_masspercent_lodders09 / scale_percent_metallicity
    o_kg = M_envelope * init_metallicity * O_masspercent_lodders09 / scale_percent_metallicity
    si_kg = M_envelope * init_metallicity * Si_masspercent_lodders09 / scale_percent_metallicity
    
    # Palme and O'Neill (2014) Treatise on Geochemistry - Table 3
    o_kg_magma = M_core * O_massfraction_palme14
    si_kg_magma = M_core * Si_massfraction_palme14

    mass_constraints = {
        "H": h_kg,
        "He": he_kg,
#        "C": c_kg,
#        "N": n_kg,
        "O": o_kg + o_kg_magma,
        "Si": si_kg + si_kg_magma,
    }

    # Magma - No solubility - Ideal Gas
    model_magma_nosol_ideal.solve(
        state=planet,
        mass_constraints=mass_constraints,
        solver_type="basic",
    )
    output_magma_nosol_ideal = model_magma_nosol_ideal.output
    # output_magma_nosol_ideal.quick_look()
    output_magma_nosol_ideal.to_excel(
        f"HHeCNOSi_magma_nosol_ideal_{init_metallicity}xsolar"
    )

    # Magma - Solubility - Ideal Gas
    initial_log_number_moles = output_magma_nosol_ideal.log_number_moles
    model_magma_sol_ideal.solve(
        state=planet,
        mass_constraints=mass_constraints,
        solver_type="basic",
        initial_log_number_moles=initial_log_number_moles,
    )
    output_magma_sol_ideal = model_magma_sol_ideal.output
    # output_magma_sol_ideal.quick_look()
    output_magma_sol_ideal.to_excel(
        f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar"
    )

    # Magma - Solubility - Real Gas
    initial_log_number_moles = output_magma_sol_ideal.log_number_moles
    model_magma_sol_real.solve(
        state=planet,
        mass_constraints=mass_constraints,
        solver_type="basic",
        initial_log_number_moles=initial_log_number_moles,
    )
    output_magma_sol_real = model_magma_sol_real.output
    output_magma_sol_real.quick_look()
    output_magma_sol_real.to_excel(
        f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar"
    )

[12:57:06 - atmodeller.classes             - INFO     ] - species_network = ('H2O_g: IdealGas, NoSolubility', 'H2_g: IdealGas, NoSolubility', 'O2_g: IdealGas, NoSolubility', 'OSi_g: IdealGas, NoSolubility', 'H4Si_g: IdealGas, NoSolubility', 'N2_g: IdealGas, NoSolubility', 'H3N_g: IdealGas, NoSolubility', 'O2Si_l: CondensateActivity, NoSolubility', 'He_g: IdealGas, NoSolubility')
[12:57:06 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 1996 K and 6000 K
[12:57:06 - atmodeller.classes             - INFO     ] - reactions = {0: '1.3333333333333333 H2O_g + 0.6666666666666667 OSi_g = 1.0 O2_g + '
    '0.6666666666666666 H4Si_g',
 1: '0.5 H2O_g + 0.5 H4Si_g + 0.5 N2_g = 0.5 OSi_g + 1.0 H3N_g',
 2: '0.6666666666666666 H2O_g + 1.3333333333333335 OSi_g = 0.3333333333333333 '
    'H4Si_g + 1.0 O2Si_l',
 3: '0.33333333333333337 H2O_g + 0.3333333333333333 H4Si_g = 1.0 H2_g + '
    '0.33333333333333337 OSi_g'}
[12:57:06 - atmodeller.classes         

OverflowError: cannot convert float infinity to integer

In [5]:
M_envelope = np.logspace(-3,-2, 2) * M_uranus

# Rocky Core 
M_core = M_uranus - M_envelope
core_mantle_melt_fraction = 1.0
R_core = 1.02 * (M_core / M_earth)**0.252 * R_earth # M-R relation from Hakim et al. (2018)
print(R_core/R_earth)
core_temperature = 2000

[2.001828155204701 1.997268070634004]


In [6]:
# Overall structure 
R_earth = 6.371e6
R_uranus = 3.939 * R_earth
M_earth = 5.972e24 #kg
M_uranus = 14.536 * M_earth

# Bulk Envelope
M_envelope = np.logspace(-3,-2, 10) * M_uranus

# Rocky Core 
M_core = M_uranus - M_envelope
core_mantle_melt_fraction = 1.0
R_core = 1.02 * (M_core / M_earth)**0.252 * R_earth # M-R relation from Hakim et al. (2018)
core_temperature = 3000

#Trying out iteration - First do the 1 metallicity case
init_metallicity = 1
planet = Planet(
    temperature=core_temperature,
    planet_mass=M_core,
    mantle_melt_fraction=1,
    surface_radius=R_core,
)

# Lodders et al. (2009) Springer book chapter - Table 8
scale_percent_metallicity = H_masspercent_lodders09 + He_masspercent_lodders09 + init_metallicity * (
    C_masspercent_lodders09 + N_masspercent_lodders09 + 
    Si_masspercent_lodders09 + 
    O_masspercent_lodders09)

h_kg = M_envelope * H_masspercent_lodders09 / scale_percent_metallicity
he_kg = M_envelope * He_masspercent_lodders09 / scale_percent_metallicity
c_kg = M_envelope * init_metallicity * C_masspercent_lodders09 / scale_percent_metallicity
n_kg = M_envelope * init_metallicity * N_masspercent_lodders09 / scale_percent_metallicity
o_kg = M_envelope * init_metallicity * O_masspercent_lodders09 / scale_percent_metallicity
si_kg = M_envelope * init_metallicity * Si_masspercent_lodders09 / scale_percent_metallicity
    
# Palme and O'Neill (2014) Treatise on Geochemistry - Table 3
o_kg_magma = M_core * O_massfraction_palme14
si_kg_magma = M_core * Si_massfraction_palme14

mass_constraints = {
    "H": h_kg,
    "He": he_kg,
    "C": c_kg,
    "N": n_kg,
    "O": o_kg + o_kg_magma,
    "Si": si_kg + si_kg_magma,
}

# Magma - No solubility - Ideal Gas
model_magma_nosol_ideal.solve(
    state=planet,
    mass_constraints=mass_constraints,
    solver_type="basic",
)
output_magma_nosol_ideal = model_magma_nosol_ideal.output
# output_magma_nosol_ideal.quick_look()
output_magma_nosol_ideal.to_excel(
    f"HHeCNOSi_magma_nosol_ideal_{init_metallicity}xsolar"
)

# Magma - Solubility - Ideal Gas
initial_log_number_moles = output_magma_nosol_ideal.log_number_moles
model_magma_sol_ideal.solve(
    state=planet,
    mass_constraints=mass_constraints,
    solver_type="basic",
    initial_log_number_moles=initial_log_number_moles,
)
output_magma_sol_ideal = model_magma_sol_ideal.output
# output_magma_sol_ideal.quick_look()
output_magma_sol_ideal.to_excel(
    f"HHeCNOSi_magma_sol_ideal_{init_metallicity}xsolar"
)

# Magma - Solubility - Real Gas
initial_log_number_moles = output_magma_sol_ideal.log_number_moles
model_magma_sol_real.solve(
    state=planet,
    mass_constraints=mass_constraints,
    solver_type="basic",
    initial_log_number_moles=initial_log_number_moles,
)
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel(
    f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar"
)



init_metallicitys = [1, 3, 5, 10, 30, 100]
for init_metallicity in init_metallicitys:

    planet = Planet(
        temperature=core_temperature,
        planet_mass=M_core,
        mantle_melt_fraction=1,
        surface_radius=R_core,
    )

    # Lodders et al. (2009) Springer book chapter - Table 8
    scale_percent_metallicity = H_masspercent_lodders09 + He_masspercent_lodders09 + init_metallicity * (
        C_masspercent_lodders09 + N_masspercent_lodders09 + 
        Si_masspercent_lodders09 + 
        O_masspercent_lodders09)

    h_kg = M_envelope * H_masspercent_lodders09 / scale_percent_metallicity
    he_kg = M_envelope * He_masspercent_lodders09 / scale_percent_metallicity
    c_kg = M_envelope * init_metallicity * C_masspercent_lodders09 / scale_percent_metallicity
    n_kg = M_envelope * init_metallicity * N_masspercent_lodders09 / scale_percent_metallicity
    o_kg = M_envelope * init_metallicity * O_masspercent_lodders09 / scale_percent_metallicity
    si_kg = M_envelope * init_metallicity * Si_masspercent_lodders09 / scale_percent_metallicity
    
    # Palme and O'Neill (2014) Treatise on Geochemistry - Table 3
    o_kg_magma = M_core * O_massfraction_palme14
    si_kg_magma = M_core * Si_massfraction_palme14

    mass_constraints = {
        "H": h_kg,
        "He": he_kg,
        "C": c_kg,
        "N": n_kg,
        "O": o_kg + o_kg_magma,
        "Si": si_kg + si_kg_magma,
    }

    # Magma - Solubility - Real Gas
    initial_log_number_moles = output_magma_sol_real.log_number_moles
    model_magma_sol_real.solve(
        state=planet,
        mass_constraints=mass_constraints,
        solver_type="basic",
        initial_log_number_moles=initial_log_number_moles,
    )
    output_magma_sol_real = model_magma_sol_real.output
    output_magma_sol_real.quick_look()
    output_magma_sol_real.to_excel(
        f"HHeCNOSi_magma_sol_real_{init_metallicity}xsolar"
    )


[12:33:52 - atmodeller.classes             - INFO     ] - Solve (basic) complete: 10 (100.00%) successful model(s)
[12:33:52 - atmodeller.classes             - INFO     ] - Multistart summary: 10 (100.00%) models(s) required 1 attempt(s)
[12:33:52 - atmodeller.classes             - INFO     ] - Solver steps (max) = 36
[12:33:52 - atmodeller.output_core         - DEBUG    ] - Creating Output
[12:33:52 - atmodeller.output              - INFO     ] - Writing output to excel
[12:33:52 - atmodeller.output_core         - INFO     ] - Computing to_dataframes output
[12:33:52 - atmodeller.output_core         - INFO     ] - Computing asdict output
[12:33:57 - atmodeller.output_core         - DEBUG    ] - Found O2_g so back-computing log10 shift for fO2
[12:34:01 - atmodeller.output              - INFO     ] - Output written to HHeCNOSi_magma_nosol_ideal_1xsolar.xlsx
[12:34:01 - atmodeller.classes             - INFO     ] - Solve (basic) complete: 10 (100.00%) successful model(s)
[12:34:01 - atm

EquinoxRuntimeError: Above is the stack outside of JIT. Below is the stack inside of JIT:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "/home/mathias/anaconda3/lib/python3.12/asyncio/base_events.py", line 639, in run_forever
    self._run_once()
  File "/home/mathias/anaconda3/lib/python3.12/asyncio/base_events.py", line 1985, in _run_once
    handle

# Case 1

In [7]:
#Overall structure 
R_earth = 6.371e6
R_uranus = 2.5388e7 #meters
M_earth = 5.972e24 #kg
M_uranus = 14.536*M_earth

#Boundary inner and outer envelope
M_until_inner_envelope = 0.913*M_uranus 
R_inner_envelope = 0.772*R_uranus
inner_envelope_temperature = 2338.8
inner_en_core_mass_fraction = 0
boundary_mantle_melt_fraction = 0

#Core 
M_core = 0.61*M_earth 
core_mass_fraction_U = 0
core_mantle_melt_fraction = 1.0
R_core = 0.7*R_earth 
core_temperature = 6000

M_heavy = 12.4*M_earth

Z_1 = 0.17
Z_2 = 0.915
Z_solar = 0.0142
print(Z_2/Z_solar)
ice_rock_ratio = 19

64.43661971830986


In [8]:
M_inner_envelope = M_until_inner_envelope - M_core
M_outer_envelope = M_uranus - M_until_inner_envelope

M_rock = M_heavy / (1+ice_rock_ratio)
M_ice = M_heavy - M_rock 

M_H_He_outer_envelope = (1-Z_1)*M_outer_envelope
M_H_He_inner_envelope = (1-Z_2)*M_inner_envelope
M_H_He_envelope = M_H_He_outer_envelope + M_H_He_inner_envelope
print(M_H_He_envelope / M_earth)
print(M_outer_envelope/ M_earth)
print(M_inner_envelope/ M_earth)

2.12586084
1.2646320000000006
12.661368


In [9]:
#SiO2 for primitive composition of Earth
SiO2_mantlemasspercent_palme14: float = 45.4
total_mantlemasspercent_palme14: float = 98.41
Si_massfraction_palme14: float = round(
    SiO2_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (28.0855 + 2 * 15.999)
    * 28.0855,
    3,
)

O_massfraction_palme141: float = Si_massfraction_palme14 / 28.0855 * 2 * 15.999

MgO_mantlemasspercent_palme14: float = 36.77
Mg_massfraction_palme14: float = round(
    MgO_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (24.305 + 15.999)
    * 24.305,
    3,
)
O_massfraction_palme142: float = Mg_massfraction_palme14 / 24.305 * 15.999
O_massfraction_palme14 = O_massfraction_palme141 + O_massfraction_palme142
print(O_massfraction_palme14)
print(Si_massfraction_palme14)

0.394198674005231
0.216


In [10]:
Z_CH4 = 0.31
Z_NH3 = 0.08
Z_H2O = np.linspace(0.6,1, 100)
Z_H2O = 0.61


print(Z_H2O)
o_kgs_perc = Z_H2O *  15.999 / ( 15.999 + 2 * 1.00794)
h_kgs_perc = Z_H2O *  2 * 1.00794 / ( 15.999 + 2 * 1.00794) + Z_NH3 * 3 * 1.00794 / ( 14.0067 + 3 * 1.00794) #+ Z_CH4 * 4 * 1.00794 / ( 12.0107 + 4 * 1.00794)
n_kgs_perc = Z_NH3 * 14.0067 / ( 14.0067 + 3 * 1.00794)
c_kgs_perc = Z_CH4 * 12.0107 / ( 12.0107 + 4 * 1.00794)

o_kgs_ice = o_kgs_perc*M_ice
h_kgs_ice = h_kgs_perc*M_ice
n_kgs_ice = n_kgs_perc*M_ice
c_kgs_ice = c_kgs_perc*M_ice

print(o_kgs_perc)
print(h_kgs_perc)
print(n_kgs_perc)
print(c_kgs_perc)

0.61
0.5417404945245263
0.08246374586273131
0.0657957596127423
0.23209139994738961


In [11]:
M_H = h_kgs_ice + 0.725*M_H_He_envelope
M_He = 0.275*M_H_He_envelope
M_O = o_kgs_ice + M_rock*O_massfraction_palme14
M_Si = M_rock*Si_massfraction_palme14
M_Mg = M_rock*Mg_massfraction_palme14
M_C = c_kgs_ice
M_N = n_kgs_ice
print(M_H)
print(M_He)
print(M_O)
print(M_Si)
print(M_Mg)
print(M_C)
print(M_N)

1.5005677394590487e+25
3.491301257532001e+24
3.957110624659829e+25
7.9977024e+23
8.33094e+23
1.6327667120922852e+25
4.6287422160779586e+24


In [12]:
eos_models = get_eos_models()
sol_models = get_solubility_models()

H2O_g = ChemicalSpecies.create_gas("H2O")
H2O_gs = ChemicalSpecies.create_gas("H2O", solubility=sol_models["H2O_peridotite_sossi23"])
H2O_rgs = ChemicalSpecies.create_gas(
    "H2O",
    activity=eos_models["H2O_cork_holland98"],
    solubility=sol_models["H2O_peridotite_sossi23"],
)

H2_g = ChemicalSpecies.create_gas("H2")
H2_gs = ChemicalSpecies.create_gas("H2", solubility=sol_models["H2_basalt_hirschmann12"])
H2_rgs = ChemicalSpecies.create_gas(
    "H2", activity=eos_models["H2_chabrier21"], solubility=sol_models["H2_basalt_hirschmann12"]
)

O2_g = ChemicalSpecies.create_gas("O2")
O2_rg = ChemicalSpecies.create_gas("O2", activity=eos_models["O2_cs_shi92"])

OSi_g = ChemicalSpecies.create_gas("OSi")
OSi_rg = ChemicalSpecies.create_gas("OSi", activity=eos_models["OSi_rk49_connolly16"])

H4Si_g = ChemicalSpecies.create_gas("H4Si")
H4Si_rg = ChemicalSpecies.create_gas("H4Si", activity=eos_models["H4Si_wang18"])

O2Si_l = ChemicalSpecies.create_condensed("O2Si", state="l")
O2Si_bqz = ChemicalSpecies.create_condensed("O2Si", state="bqz")
CO2_g = ChemicalSpecies.create_gas("CO2")
CO2_gs = ChemicalSpecies.create_gas("CO2", solubility=sol_models["CO2_basalt_dixon95"])
CO2_rgs = ChemicalSpecies.create_gas(
    "CO2", solubility=sol_models["CO2_basalt_dixon95"], activity=eos_models["CO2_cs_shi92"]
)
#
CO_g = ChemicalSpecies.create_gas("CO")
CO_gs = ChemicalSpecies.create_gas("CO", solubility=sol_models["CO_basalt_yoshioka19"])
CO_rgs = ChemicalSpecies.create_gas(
    "CO", solubility=sol_models["CO_basalt_yoshioka19"], activity=eos_models["CO_cs_shi92"]
)
#
CH4_g = ChemicalSpecies.create_gas("CH4")
CH4_gs = ChemicalSpecies.create_gas("CH4", solubility=sol_models["CH4_basalt_ardia13"])
CH4_rgs = ChemicalSpecies.create_gas(
    "CH4", solubility=sol_models["CH4_basalt_ardia13"], activity=eos_models["CH4_cs_shi92"]
)
#
N2_g = ChemicalSpecies.create_gas("N2")
N2_gs = ChemicalSpecies.create_gas("N2", solubility=sol_models["N2_basalt_libourel03"])
N2_rgs = ChemicalSpecies.create_gas(
    "N2",solubility=sol_models["N2_basalt_libourel03"],activity=eos_models["N2_cs_saxena87"]
)

NH3_g = ChemicalSpecies.create_gas("NH3")
NH3_rg = ChemicalSpecies.create_gas("H3N", activity=eos_models["H3N_rk49_reid87"])

He_g = ChemicalSpecies.create_gas("He")
He_gs = ChemicalSpecies.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
He_rgs = ChemicalSpecies.create_gas(
    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
)

Mg_g = ChemicalSpecies.create_gas("Mg")
#Mg_gs = ChemicalSpecies.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
#Mg_rgs = Species.create_gas(
#    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
#)

MgO_l = ChemicalSpecies.create_condensed("MgO", state="l")
MgO_cr = ChemicalSpecies.create_condensed("MgO", state="cr")
# MgH_g = ChemicalSpecies.create_gas("MgH")
# MgOH_g = ChemicalSpecies.create_gas("MgOH")

species_HHeCNOSi_magma_nosol_ideal = SpeciesNetwork(
    (H2O_g, H2_g, O2_g, OSi_g, H4Si_g,N2_g, NH3_g, O2Si_l, He_g)
     #)
)
species_HHeCNOSi_magma_sol_ideal = SpeciesNetwork(
    (H2O_gs, H2_gs, O2_g, OSi_g, H4Si_g,N2_gs, NH3_g, O2Si_l, He_gs)
     #, )
)
species_HHeCNOSi_magma_sol_real = SpeciesNetwork(
    (
        H2O_rgs,
        H2_rgs,
        O2_rg,
        OSi_rg,
        H4Si_rg,
        O2Si_l,
#        CO2_rgs,
#        CO_rgs,
#        CH4_rgs,
        N2_rgs,
        NH3_rg,
        He_rgs
    )
)

In [13]:
mass_constraints = {
    "H": M_H,
    "He": M_He,
#    "C": M_C,
    "N": M_N,
    "Si":  M_Si,
    "O": M_O,
#    "Mg": metall * mg_kgs_solar + core_mantle_melt_fraction * mg_kg_magma,
        }
#solver = optx.LevenbergMarquardt, 
solver_parameters = SolverParameters(multistart= 50)
Uranus = Planet(
    temperature=4500,
    planet_mass=M_core,
    mantle_melt_fraction=core_mantle_melt_fraction,
    core_mass_fraction = core_mass_fraction_U,
    surface_radius=R_core,
    )

model_magma_nosol_ideal = EquilibriumModel(species_HHeCNOSi_magma_nosol_ideal)
model_magma_nosol_ideal.solve(
        state=Uranus,
        mass_constraints=mass_constraints,
        )
output_magma_nosol_ideal = model_magma_nosol_ideal.output
output_magma_nosol_ideal.quick_look()
output_magma_nosol_ideal.to_excel("HHeCNOSi_magma_nosol_ideal")

model_magma_sol_ideal = EquilibriumModel(species_HHeCNOSi_magma_sol_ideal)
initial_log_number_moles = output_magma_nosol_ideal.log_number_moles
model_magma_sol_ideal.solve(
        state=Uranus,
        mass_constraints=mass_constraints,
        initial_log_number_moles=initial_log_number_moles,
        )
output_magma_sol_ideal = model_magma_sol_ideal.output
output_magma_sol_ideal.quick_look()
output_magma_sol_ideal.to_excel("HHeCNOSi_magma_sol_ideal")

#last minute solution, reverting to other solver 
model_magma_sol_real = EquilibriumModel(species_HHeCNOSi_magma_sol_real)
initial_log_number_moles = output_magma_sol_ideal.log_number_moles
model_magma_sol_real.solve(
        state=Uranus,
        mass_constraints=mass_constraints,
        solver_parameters= solver_parameters,
        initial_log_number_moles=initial_log_number_moles
        )
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel("Core_HHeSiO_Testnew")

[12:44:15 - atmodeller.classes             - INFO     ] - species_network = ('H2O_g: IdealGas, NoSolubility', 'H2_g: IdealGas, NoSolubility', 'O2_g: IdealGas, NoSolubility', 'OSi_g: IdealGas, NoSolubility', 'H4Si_g: IdealGas, NoSolubility', 'N2_g: IdealGas, NoSolubility', 'H3N_g: IdealGas, NoSolubility', 'O2Si_l: CondensateActivity, NoSolubility', 'He_g: IdealGas, NoSolubility')
[12:44:15 - atmodeller.classes             - INFO     ] - Thermodynamic data requires temperatures between 1996 K and 6000 K
[12:44:15 - atmodeller.classes             - INFO     ] - reactions = {0: '1.3333333333333333 H2O_g + 0.6666666666666667 OSi_g = 1.0 O2_g + '
    '0.6666666666666666 H4Si_g',
 1: '0.5 H2O_g + 0.5 H4Si_g + 0.5 N2_g = 0.5 OSi_g + 1.0 H3N_g',
 2: '0.6666666666666666 H2O_g + 1.3333333333333335 OSi_g = 0.3333333333333333 '
    'H4Si_g + 1.0 O2Si_l',
 3: '0.33333333333333337 H2O_g + 0.3333333333333333 H4Si_g = 1.0 H2_g + '
    '0.33333333333333337 OSi_g'}
[12:44:31 - atmodeller.classes         

EquinoxRuntimeError: Above is the stack outside of JIT. Below is the stack inside of JIT:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/mathias/atmodeller/.venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "/home/mathias/anaconda3/lib/python3.12/asyncio/base_events.py", line 639, in run_forever
    self._run_once()
  File "/home/mathias/anaconda3/lib/python3.12/asyncio/base_events.py", line 1985, in _run_once
    handle

In [ ]:
H2_pressure_magma_sol_real = []
H2O_pressure_magma_sol_real = []
O2_pressure_magma_sol_real = []
SiH4_pressure_magma_sol_real = []
SiO_pressure_magma_sol_real = []
He_pressure_magma_sol_real = []
tot_pressure_magma_sol_real = []

H_moles_atm_magma_sol_real = []
O_moles_atm_magma_sol_real = []
Si_moles_atm_magma_sol_real = []
He_moles_atm_magma_sol_real = []

H_moles_melt_magma_sol_real = []
O_moles_melt_magma_sol_real = []
Si_moles_melt_magma_sol_real = []
He_moles_melt_magma_sol_real = []

H_moles_tot_magma_sol_real = []
O_moles_tot_magma_sol_real = []
Si_moles_tot_magma_sol_real = []
He_moles_tot_magma_sol_real = []

H_mass_atm_magma_sol_real = []
O_mass_atm_magma_sol_real = []
Si_mass_atm_magma_sol_real = []
He_mass_atm_magma_sol_real = []

H_mass_melt_magma_sol_real = []
O_mass_melt_magma_sol_real = []
Si_mass_melt_magma_sol_real = []
He_mass_melt_magma_sol_real = []

H_mass_tot_magma_sol_real = []
O_mass_tot_magma_sol_real = []
Si_mass_tot_magma_sol_real = []
He_mass_tot_magma_sol_real = []

In [ ]:
Temperatures = list(range(3200, 4651, 50))
for T in Temperatures:
    filename = f"Core_HHeSiO_{Temperature}.xlsx"

    H2_pressure_magma_sol_real.append(pd.read_excel(filename, sheet_name="H2_g")["pressure"])
    H2O_pressure_magma_sol_real.append(pd.read_excel(filename, sheet_name="H2O_g")["pressure"])
    O2_pressure_magma_sol_real.append(pd.read_excel(filename, sheet_name="O2_g")["pressure"])
    SiH4_pressure_magma_sol_real.append(pd.read_excel(filename, sheet_name="H4Si_g")["pressure"])
    SiO_pressure_magma_sol_real.append(pd.read_excel(filename, sheet_name="OSi_g")["pressure"])
    He_pressure_magma_sol_real.append(pd.read_excel(filename, sheet_name="He_g")["pressure"])
    tot_pressure_magma_sol_real.append(pd.read_excel(filename, sheet_name="atmosphere")["pressure"])

    H_moles_atm_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_H")["atmosphere_moles"])
    O_moles_atm_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_O")["atmosphere_moles"])
    Si_moles_atm_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_Si")["atmosphere_moles"])
    He_moles_atm_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_He")["atmosphere_moles"])

    H_moles_melt_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_H")["dissolved_moles"])
    O_moles_melt_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_O")["dissolved_moles"])
    Si_moles_melt_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_Si")["dissolved_moles"])
    He_moles_melt_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_He")["dissolved_moles"])

    H_moles_tot_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_H")["total_moles"])
    O_moles_tot_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_O")["total_moles"])
    Si_moles_tot_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_Si")["total_moles"])
    He_moles_tot_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_He")["total_moles"])

    H_mass_atm_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"])
    O_mass_atm_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"])
    Si_mass_atm_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"])
    He_mass_atm_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"])

    H_mass_melt_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_H")["dissolved_mass"])
    O_mass_melt_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_O")["dissolved_mass"])
    Si_mass_melt_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_Si")["dissolved_mass"])
    He_mass_melt_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_He")["dissolved_mass"])

    H_mass_tot_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_H")["total_mass"])
    O_mass_tot_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_O")["total_mass"])
    Si_mass_tot_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_Si")["total_mass"])
    He_mass_tot_magma_sol_real.append(pd.read_excel(filename, sheet_name="element_He")["total_mass"])

In [ ]:
color_tot = "black"
color_H2 = "orange"
color_H2O = "blue"
color_SiH4 = "red"
color_SiO = "brown"
color_O2 = "limegreen"
color_CO2 = "cyan"
color_CO = "magenta"
color_CH4 = "purple"
color_N2 = "green"
color_NH3 = "pink"
color_MgO = "darkred"
color_He = "gray"
color_H = "orange"
color_C = "cyan"
color_N = "green"
color_O = "blue"
color_Si = "red"
color_Mg = "lime"
color_MgH2 = "gold"
color_MgH = "teal"
color_MgOH = "indigo"

In [ ]:
fig_big, axes = plt.subplots(6, 5, figsize=(15, 6 * 5), sharey='row')
axes = axes.flatten()
legend_handles = None
legend_labels = None
Hmps = np.array(H_mass_tot_magma_sol_real) / M_uranus 
for i in range(len(Temperatures)):

    fig, ax = plt.subplots(1, 1, figsize=(15, 20))

    ax.plot(Hmps[i], tot_pressure_magma_sol_real[i], color=color_tot, lw=4, ls="-", label="Total")
    ax.plot(Hmps[i], H2_pressure_magma_sol_real[i], color=color_H2, lw=3, ls="-", label="H$_2$")
    ax.plot(Hmps[i], H2O_pressure_magma_sol_real[i], color=color_H2O, lw=3, ls="-", label="H$_2$O")
    ax.plot(Hmps[i], SiH4_pressure_magma_sol_real[i], color=color_SiH4, lw=3, ls="-", label="SiH$_4$")
    ax.plot(Hmps[i], SiO_pressure_magma_sol_real[i], color=color_SiO, lw=3, ls="-", label="SiO")
    ax.plot(Hmps[i], He_pressure_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    ax.plot(Hmps[i], tot_pressure_magma_sol_real[i], color=color_tot, lw=3, ls="-", label="tot")

    ax.set_title(r"(a) Real gas")
    ax.set_ylim([1e-3, 2e6])
    ax.set_yscale("log")
    ax.set_xlabel(r"Hmps", fontsize=10)
    ax.set_ylabel(r"MEB Partial Pressure [bar]", fontsize=12)
    ax.get_xaxis().set_major_formatter(ScalarFormatter())

    ax.legend(fontsize=10, loc='best') 
        
    plt.tight_layout()
    title = f"Uranus Interior: Temperature core boundary{T:.0f}"
    filename = f"./Test_constraints_images/uranus_{T:.0f}_MEB_pressures.png"

    plt.title(title)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close(fig)

    axes[i].plot(Hmps[i], tot_pressure_magma_sol_real[i], color=color_tot, lw=4, ls="-", label="Total")
    axes[i].plot(Hmps[i], H2_pressure_magma_sol_real[i], color=color_H2, lw=3, ls="-", label="H$_2$")
    axes[i].plot(Hmps[i], H2O_pressure_magma_sol_real[i], color=color_H2O, lw=3, ls="-", label="H$_2$O")
    axes[i].plot(Hmps[i], SiH4_pressure_magma_sol_real[i], color=color_SiH4, lw=3, ls="-", label="SiH$_4$")
    axes[i].plot(Hmps[i], SiO_pressure_magma_sol_real[i], color=color_SiO, lw=3, ls="-", label="SiO")
    axes[i].plot(Hmps[i], He_pressure_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")

    axes[i].set_title(f"T ≈ {Temperatures[i]:.2f} ")
    axes[i].set_ylim([1e-3, 2e6])
    axes[i].set_yscale("log")
    axes[i].set_xlabel(r"Hmps", fontsize=10)
    axes[i].set_ylabel(r"MEB Partial Pressure [bar]", fontsize=12)
    axes[i].get_xaxis().set_major_formatter(ScalarFormatter())
        # Capture the legend only once
    if legend_handles is None:
        legend_handles, legend_labels = axes[i].get_legend_handles_labels()
    axes[i].legend().remove() 
        
fig_big.subplots_adjust(left=0.15)

# Plot ONE legend on the left side, centered vertically
fig_big.legend(
    legend_handles,
    legend_labels,
    loc='center left',
    bbox_to_anchor=(0, 0.5)   # x-position, y-position
)

fig_big.suptitle(f"Uranus Interior: :, MEB pressures ", fontsize=16, y=0.95) 
fig_big.savefig(f"./Test_constraints_images/uranus_MEB_pressures.png", dpi=300, bbox_inches='tight')

In [ ]:
fig_big, axes = plt.subplots(6, 5, figsize=(15, 6 * 5), sharey='row')
axes = axes.flatten()
legend_handles = None
legend_labels = None
Hmps = np.array(H_mass_tot_magma_sol_real) / M_uranus 
for i in range(len(Temperatures)):

    fig, ax = plt.subplots(1, 1, figsize=(15, 20))

    ax.plot(Hmps[i], H_moles_atm_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    ax.plot(Hmps[i], He_moles_atm_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    ax.plot(Hmps[i], Si_moles_atm_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    ax.plot(Hmps[i], O_moles_atm_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    ax.set_title(r"(a) Real gas")
    #ax.set_ylim([1e-3, 2e6])
    ax.set_yscale("log")
    ax.set_xlabel(r"Hmps", fontsize=10)
    ax.set_ylabel(r"Atmosphere moles", fontsize=12)
    ax.get_xaxis().set_major_formatter(ScalarFormatter())

    ax.legend(fontsize=10, loc='best') 
        
    plt.tight_layout()
    title = f"Uranus Interior: Temperature core boundary{T:.0f}"
    filename = f"./Test_constraints_images/uranus_{T:.0f}_Atmosphere_moles.png"

    plt.title(title)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close(fig)

    axes[i].plot(Hmps[i], H_moles_atm_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    axes[i].plot(Hmps[i], He_moles_atm_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    axes[i].plot(Hmps[i], Si_moles_atm_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    axes[i].plot(Hmps[i], O_moles_atm_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    axes[i].set_title(f"T ≈ {Temperatures[i]:.2f} ")
    #axes[i].set_ylim([1e-3, 2e6])
    axes[i].set_yscale("log")
    axes[i].set_xlabel(r"Hmps", fontsize=10)
    axes[i].set_ylabel(r"Atmosphere moles", fontsize=12)
    axes[i].get_xaxis().set_major_formatter(ScalarFormatter())
        # Capture the legend only once
    if legend_handles is None:
        legend_handles, legend_labels = axes[i].get_legend_handles_labels()
    axes[i].legend().remove() 
        
fig_big.subplots_adjust(left=0.15)

# Plot ONE legend on the left side, centered vertically
fig_big.legend(
    legend_handles,
    legend_labels,
    loc='center left',
    bbox_to_anchor=(0, 0.5)   # x-position, y-position
)

fig_big.suptitle(f"Uranus Interior: :, Atmosphere moles ", fontsize=16, y=0.95) 
fig_big.savefig(f"./Test_constraints_images/uranus_atmo_moles.png", dpi=300, bbox_inches='tight')

In [ ]:
fig_big, axes = plt.subplots(6, 5, figsize=(15, 6 * 5), sharey='row')
axes = axes.flatten()
legend_handles = None
legend_labels = None
Hmps = np.array(H_mass_tot_magma_sol_real) / M_uranus 
for i in range(len(Temperatures)):

    fig, ax = plt.subplots(1, 1, figsize=(15, 20))

    ax.plot(Hmps[i], H_moles_melt_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    ax.plot(Hmps[i], He_moles_melt_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    ax.plot(Hmps[i], Si_moles_melt_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    ax.plot(Hmps[i], O_moles_melt_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    ax.set_title(r"(a) Real gas")
#    ax.set_ylim([1e-3, 2e6])
    ax.set_yscale("log")
    ax.set_xlabel(r"Hmps", fontsize=10)
    ax.set_ylabel(r"Dissolved moles", fontsize=12)
    ax.get_xaxis().set_major_formatter(ScalarFormatter())

    ax.legend(fontsize=10, loc='best') 
        
    plt.tight_layout()
    title = f"Uranus Interior: Temperature core boundary{T:.0f}"
    filename = f"./Test_constraints_images/uranus_{T:.0f}_dissolved_moles.png"

    plt.title(title)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close(fig)

    axes[i].plot(Hmps[i], H_moles_melt_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    axes[i].plot(Hmps[i], He_moles_melt_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    axes[i].plot(Hmps[i], Si_moles_melt_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    axes[i].plot(Hmps[i], O_moles_melt_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    axes[i].set_title(f"T ≈ {Temperatures[i]:.2f} ")
#    axes[i].set_ylim([1e-3, 2e6])
    axes[i].set_yscale("log")
    axes[i].set_xlabel(r"Hmps", fontsize=10)
    axes[i].set_ylabel(r"Dissolved moles", fontsize=12)
    axes[i].get_xaxis().set_major_formatter(ScalarFormatter())
        # Capture the legend only once
    if legend_handles is None:
        legend_handles, legend_labels = axes[i].get_legend_handles_labels()
    axes[i].legend().remove() 
        
fig_big.subplots_adjust(left=0.15)

# Plot ONE legend on the left side, centered vertically
fig_big.legend(
    legend_handles,
    legend_labels,
    loc='center left',
    bbox_to_anchor=(0, 0.5)   # x-position, y-position
)

fig_big.suptitle(f"Uranus Interior: :, Dissolved moles ", fontsize=16, y=0.95) 
fig_big.savefig(f"./Test_constraints_images/uranus_melt_moles.png", dpi=300, bbox_inches='tight')

In [ ]:
fig_big, axes = plt.subplots(6, 5, figsize=(15, 6 * 5), sharey='row')
axes = axes.flatten()
legend_handles = None
legend_labels = None
Hmps = np.array(H_mass_tot_magma_sol_real) / M_uranus 
for i in range(len(Temperatures)):

    fig, ax = plt.subplots(1, 1, figsize=(15, 20))

    ax.plot(Hmps[i], H_moles_tot_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    ax.plot(Hmps[i], He_moles_tot_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    ax.plot(Hmps[i], Si_moles_tot_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    ax.plot(Hmps[i], O_moles_tot_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    ax.set_title(r"(a) Real gas")
#    ax.set_ylim([1e-3, 2e6])
    ax.set_yscale("log")
    ax.set_xlabel(r"Hmps", fontsize=10)
    ax.set_ylabel(r"Total moles", fontsize=12)
    ax.get_xaxis().set_major_formatter(ScalarFormatter())

    ax.legend(fontsize=10, loc='best') 
        
    plt.tight_layout()
    title = f"Uranus Interior: Temperature core boundary{T:.0f}"
    filename = f"./Test_constraints_images/uranus_{T:.0f}_total_moles.png"

    plt.title(title)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close(fig)

    axes[i].plot(Hmps[i], H_moles_tot_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    axes[i].plot(Hmps[i], He_moles_tot_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    axes[i].plot(Hmps[i], Si_moles_tot_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    axes[i].plot(Hmps[i], O_moles_tot_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    axes[i].set_title(f"T ≈ {Temperatures[i]:.2f} ")
#    axes[i].set_ylim([1e-3, 2e6])
    axes[i].set_yscale("log")
    axes[i].set_xlabel(r"Hmps", fontsize=10)
    axes[i].set_ylabel(r"Total moles", fontsize=12)
    axes[i].get_xaxis().set_major_formatter(ScalarFormatter())
        # Capture the legend only once
    if legend_handles is None:
        legend_handles, legend_labels = axes[i].get_legend_handles_labels()
    axes[i].legend().remove() 
        
fig_big.subplots_adjust(left=0.15)

# Plot ONE legend on the left side, centered vertically
fig_big.legend(
    legend_handles,
    legend_labels,
    loc='center left',
    bbox_to_anchor=(0, 0.5)   # x-position, y-position
)

fig_big.suptitle(f"Uranus Interior: :, Total moles ", fontsize=16, y=0.95) 
fig_big.savefig(f"./Test_constraints_images/uranus_tot_moles.png", dpi=300, bbox_inches='tight')

In [ ]:
fig_big, axes = plt.subplots(6, 5, figsize=(15, 6 * 5), sharey='row')
axes = axes.flatten()
legend_handles = None
legend_labels = None
Hmps = np.array(H_mass_tot_magma_sol_real) / M_uranus 
for i in range(len(Temperatures)):

    fig, ax = plt.subplots(1, 1, figsize=(15, 20))

    ax.plot(Hmps[i], H_mass_atm_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    ax.plot(Hmps[i], He_mass_atm_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    ax.plot(Hmps[i], Si_mass_atm_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    ax.plot(Hmps[i], O_mass_atm_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    ax.set_title(r"(a) Real gas")
    #ax.set_ylim([1e-3, 2e6])
    ax.set_yscale("log")
    ax.set_xlabel(r"Hmps", fontsize=10)
    ax.set_ylabel(r"Atmosphere mass", fontsize=12)
    ax.get_xaxis().set_major_formatter(ScalarFormatter())

    ax.legend(fontsize=10, loc='best') 
        
    plt.tight_layout()
    title = f"Uranus Interior: Temperature core boundary{Temperatures[i]:.0f}"
    filename = f"./Test_constraints_images/uranus_{Temperatures[i]:.0f}_Atmosphere_mass.png"

    plt.title(title)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close(fig)

    axes[i].plot(Hmps[i], H_mass_atm_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    axes[i].plot(Hmps[i], He_mass_atm_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    axes[i].plot(Hmps[i], Si_mass_atm_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    axes[i].plot(Hmps[i], O_mass_atm_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    axes[i].set_title(f"T ≈ {Temperatures[i]:.2f} ")
    #axes[i].set_ylim([1e-3, 2e6])
    axes[i].set_yscale("log")
    axes[i].set_xlabel(r"Hmps", fontsize=10)
    axes[i].set_ylabel(r"Atmosphere mass", fontsize=12)
    axes[i].get_xaxis().set_major_formatter(ScalarFormatter())
        # Capture the legend only once
    if legend_handles is None:
        legend_handles, legend_labels = axes[i].get_legend_handles_labels()
    axes[i].legend().remove() 
        
fig_big.subplots_adjust(left=0.15)

# Plot ONE legend on the left side, centered vertically
fig_big.legend(
    legend_handles,
    legend_labels,
    loc='center left',
    bbox_to_anchor=(0, 0.5)   # x-position, y-position
)

fig_big.suptitle(f"Uranus Interior: :, Atmosphere mass ", fontsize=16, y=0.95) 
fig_big.savefig(f"./Test_constraints_images/uranus_atmo_mass.png", dpi=300, bbox_inches='tight')

In [ ]:
fig_big, axes = plt.subplots(6, 5, figsize=(15, 6 * 5), sharey='row')
axes = axes.flatten()
legend_handles = None
legend_labels = None
Hmps = np.array(H_mass_tot_magma_sol_real) / M_uranus 
for i in range(len(Temperatures)):

    fig, ax = plt.subplots(1, 1, figsize=(15, 20))

    ax.plot(Hmps[i], H_mass_melt_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    ax.plot(Hmps[i], He_mass_melt_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    ax.plot(Hmps[i], Si_mass_melt_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    ax.plot(Hmps[i], O_mass_melt_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    ax.set_title(r"(a) Real gas")
#    ax.set_ylim([1e-3, 2e6])
    ax.set_yscale("log")
    ax.set_xlabel(r"Hmps", fontsize=10)
    ax.set_ylabel(r"Dissolved mass", fontsize=12)
    ax.get_xaxis().set_major_formatter(ScalarFormatter())

    ax.legend(fontsize=10, loc='best') 
        
    plt.tight_layout()
    title = f"Uranus Interior: Temperature core boundary{Temperatures[i]:.0f}"
    filename = f"./Test_constraints_images/uranus_{Temperatures[i]:.0f}_dissolved_mass.png"

    plt.title(title)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close(fig)

    axes[i].plot(Hmps[i], H_mass_melt_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    axes[i].plot(Hmps[i], He_mass_melt_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    axes[i].plot(Hmps[i], Si_mass_melt_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    axes[i].plot(Hmps[i], O_mass_melt_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    axes[i].set_title(f"T ≈ {Temperatures[i]:.2f} ")
#    axes[i].set_ylim([1e-3, 2e6])
    axes[i].set_yscale("log")
    axes[i].set_xlabel(r"Hmps", fontsize=10)
    axes[i].set_ylabel(r"Dissolved mass", fontsize=12)
    axes[i].get_xaxis().set_major_formatter(ScalarFormatter())
        # Capture the legend only once
    if legend_handles is None:
        legend_handles, legend_labels = axes[i].get_legend_handles_labels()
    axes[i].legend().remove() 
        
fig_big.subplots_adjust(left=0.15)

# Plot ONE legend on the left side, centered vertically
fig_big.legend(
    legend_handles,
    legend_labels,
    loc='center left',
    bbox_to_anchor=(0, 0.5)   # x-position, y-position
)

fig_big.suptitle(f"Uranus Interior: :, Dissolved mass ", fontsize=16, y=0.95) 
fig_big.savefig(f"./Test_constraints_images/uranus_melt_mass.png", dpi=300, bbox_inches='tight')

In [ ]:
fig_big, axes = plt.subplots(6, 5, figsize=(15, 6 * 5), sharey='row')
axes = axes.flatten()
legend_handles = None
legend_labels = None
Hmps = np.array(H_mass_tot_magma_sol_real) / M_uranus 
for i in range(len(Temperatures)):

    fig, ax = plt.subplots(1, 1, figsize=(15, 20))

    ax.plot(Hmps[i], H_mass_tot_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    ax.plot(Hmps[i], He_mass_tot_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    ax.plot(Hmps[i], Si_mass_tot_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    ax.plot(Hmps[i], O_mass_tot_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    ax.set_title(r"(a) Real gas")
#    ax.set_ylim([1e-3, 2e6])
    ax.set_yscale("log")
    ax.set_xlabel(r"Hmps", fontsize=10)
    ax.set_ylabel(r"Total mass", fontsize=12)
    ax.get_xaxis().set_major_formatter(ScalarFormatter())

    ax.legend(fontsize=10, loc='best') 
        
    plt.tight_layout()
    title = f"Uranus Interior: Temperature core boundary{Temperatures[i]:.0f}"
    filename = f"./Test_constraints_images/uranus_{Temperatures[i]:.0f}_total_mass.png"

    plt.title(title)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close(fig)

    axes[i].plot(Hmps[i], H_mass_tot_magma_sol_real[i], color=color_H, lw=4, ls="-", label="H")
    axes[i].plot(Hmps[i], He_mass_tot_magma_sol_real[i], color=color_He, lw=3, ls="-", label="He")
    axes[i].plot(Hmps[i], Si_mass_tot_magma_sol_real[i], color=color_Si, lw=3, ls="-", label="Si")
    axes[i].plot(Hmps[i], O_mass_tot_magma_sol_real[i], color=color_O, lw=3, ls="-", label="O")

    axes[i].set_title(f"T ≈ {Temperatures[i]:.2f} ")
#    axes[i].set_ylim([1e-3, 2e6])
    axes[i].set_yscale("log")
    axes[i].set_xlabel(r"Hmps", fontsize=10)
    axes[i].set_ylabel(r"Total moles", fontsize=12)
    axes[i].get_xaxis().set_major_formatter(ScalarFormatter())
        # Capture the legend only once
    if legend_handles is None:
        legend_handles, legend_labels = axes[i].get_legend_handles_labels()
    axes[i].legend().remove() 
        
fig_big.subplots_adjust(left=0.15)

# Plot ONE legend on the left side, centered vertically
fig_big.legend(
    legend_handles,
    legend_labels,
    loc='center left',
    bbox_to_anchor=(0, 0.5)   # x-position, y-position
)

fig_big.suptitle(f"Uranus Interior: :, Total mass ", fontsize=16, y=0.95) 
fig_big.savefig(f"./Test_constraints_images/uranus_tot_mass.png", dpi=300, bbox_inches='tight')

# Case 2

In [ ]:
#Overall structure 
R_earth = 6.371e6
R_uranus = 2.5388e7 #meters
M_earth = 5.972e24 #kg
M_uranus = 14.536*M_earth

#Boundary inner and outer envelope
M_until_inner_envelope = 0.913*M_uranus 
R_inner_envelope = 0.772*R_uranus
inner_envelope_temperature = 2338.8
inner_en_core_mass_fraction = 0
boundary_mantle_melt_fraction = 0

#Core 
M_core = 0.61*M_earth 
core_mass_fraction_U = 0
core_mantle_melt_fraction = 1.0
R_core = 0.7*R_earth 
core_temperature = 6000

M_heavy = 12.4*M_earth

Z_1 = 0.17
Z_2 = 0.915
Z_solar = 0.0142

ice_rock_ratio = 19

In [ ]:
M_inner_envelope = M_until_inner_envelope - M_core

M_rock = M_heavy / (1+ice_rock_ratio)
M_ice = M_heavy - M_rock 

M_H_He_inner_envelope = (1-Z_2)*M_inner_envelope

In [ ]:
#SiO2 for primitive composition of Earth
SiO2_mantlemasspercent_palme14: float = 45.4
total_mantlemasspercent_palme14: float = 98.41
Si_massfraction_palme14: float = round(
    SiO2_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (28.0855 + 2 * 15.999)
    * 28.0855,
    3,
)

O_massfraction_palme141: float = Si_massfraction_palme14 / 28.0855 * 2 * 15.999

MgO_mantlemasspercent_palme14: float = 36.77
Mg_massfraction_palme14: float = round(
    MgO_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (24.305 + 15.999)
    * 24.305,
    3,
)
O_massfraction_palme142: float = Mg_massfraction_palme14 / 24.305 * 15.999
O_massfraction_palme14 = O_massfraction_palme141 + O_massfraction_palme142
print(O_massfraction_palme14)
print(Si_massfraction_palme14)

In [ ]:
Z_CH4 = 0.31
Z_NH3 = 0.08
Z_H2O = 0.61

Z_H2O = np.linspace(0.6,1, 11)
o_kgs_perc = Z_H2O *  15.999 / ( 15.999 + 2 * 1.00794)
h_kgs_perc = Z_H2O *  2 * 1.00794 / ( 15.999 + 2 * 1.00794) #+ Z_NH3 * 3 * 1.00794 / ( 14.0067 + 3 * 1.00794) + Z_CH4 * 4 * 1.00794 / ( 12.0107 + 4 * 1.00794)
n_kgs_perc = Z_NH3 * 14.0067 / ( 14.0067 + 3 * 1.00794)
c_kgs_perc = Z_CH4 * 12.0107 / ( 12.0107 + 4 * 1.00794)

o_kgs_ice = o_kgs_perc*M_ice
h_kgs_ice = h_kgs_perc*M_ice
n_kgs_ice = n_kgs_perc*M_ice
c_kgs_ice = c_kgs_perc*M_ice

In [ ]:
#mass_fraction_envelope = np.logspace(0,-10,11)

mass_fraction_envelope = 0.0000000001
mass_fraction_envelope = mass_fraction_envelope#[:, None]
M_H = mass_fraction_envelope*(h_kgs_ice + 0.725*M_H_He_inner_envelope)#[None,:]
M_He = mass_fraction_envelope*0.275*M_H_He_inner_envelope
M_O = mass_fraction_envelope*o_kgs_ice + M_rock*O_massfraction_palme14
M_Si = M_rock*Si_massfraction_palme14
M_Mg = M_rock*Mg_massfraction_palme14
M_C = c_kgs_ice
M_N = n_kgs_ice

In [ ]:
mass_constraints = {
    "H": M_H,#.flatten(),
    "He": M_He,
#    "C": M_C,
#    "N": M_N,
    "Si":  M_Si,
    "O": M_O,
#    "Mg": metall * mg_kgs_solar + core_mantle_melt_fraction * mg_kg_magma,
        }
#solver = optx.LevenbergMarquardt, 
solver_parameters = SolverParameters(multistart= 50)
Uranus = Planet(
    surface_temperature=4000,
    planet_mass=M_core,
    mantle_melt_fraction=core_mantle_melt_fraction,
    core_mass_fraction = core_mass_fraction_U,
    surface_radius=R_core,
    )

model_magma_nosol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_nosol_ideal)
model_magma_nosol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        )
output_magma_nosol_ideal = model_magma_nosol_ideal.output
output_magma_nosol_ideal.quick_look()
output_magma_nosol_ideal.to_excel("HHeCNOSi_magma_nosol_ideal")

model_magma_sol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_sol_ideal)
initial_log_number_density = output_magma_nosol_ideal.log_number_density
model_magma_sol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        initial_log_number_density=initial_log_number_density,
        )
output_magma_sol_ideal = model_magma_sol_ideal.output
output_magma_sol_ideal.quick_look()
output_magma_sol_ideal.to_excel("HHeCNOSi_magma_sol_ideal")

#last minute solution, reverting to other solver 
model_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_magma_sol_real)
initial_log_number_density = output_magma_sol_ideal.log_number_density
model_magma_sol_real.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        solver_parameters= solver_parameters,
        initial_log_number_density=initial_log_number_density
        )
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel("Vertical_Test_10")